<a href="https://colab.research.google.com/github/kumaramardeep342/Colab-Work/blob/main/Ed_AI__NLP_M2%2BM3_inclass_ATIS_Intent_classification_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary - How to predict model on NLP Data
0. Load the dataset
1. Import libraries
2. Read the dataset
3. Data Preprocessing on X_data and Y_data
4. Split the dataset into X_data and Y_data
5. Split the data into training and testing
6. Use only training data to train the model
7. Hyperparameter tunning
8. pass the test to the trained model to predict y_pred_test - validation metrics
9. Create a samml API - x_data[response] - post - "airfare" etc.

# Loading the Dataset

In [335]:
! pip install -q opendatasets
import opendatasets as od
od.download('https://www.kaggle.com/datasets/hassanamin/atis-airlinetravelinformationsystem')

Skipping, found downloaded files in "./atis-airlinetravelinformationsystem" (use force=True to force download)


# About The Dataset & Model
The ATIS dataset is a standard benchmark dataset widely used as an intent classification. ATIS Stands for Airline Travel Information System. Intent classification is an important component of Natural Language Understanding (NLU) systems in any chatbot platform.

Within a chatbot, intent refers to the goal the customer has in mind when typing in a question or comment. While entity refers to the modifier the customer uses to describe their issue, the intent is what they really mean. For example, a user says, ‘I need new shoes.’ The intent behind the message is to browse the footwear on offer.


# 1.Import libraries

In [336]:
# 1. Import libraries
import pandas as pd  #for redaing  .csv dataset
import numpy as np

!pip install scikit-learn
#convert text into numerical
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

#Label Encoding
from sklearn.preprocessing import LabelEncoder
# split the dataset into train and test data
from sklearn.model_selection import train_test_split
#naive bayes classifciation
from sklearn.naive_bayes import MultinomialNB


#enable table format
from google.colab import data_table
data_table.enable_dataframe_formatter()
#disable table format
# from google.colab import data_table
# data_table.disable_dataframe_formatter()


# 2.Read the dataset

In [337]:
df=pd.read_csv('/content/atis-airlinetravelinformationsystem/atis_intents.csv')
df.columns = ['Intent','Text'] #Re-naming the columns
df.head()

,Intent,Text
0,atis_flight,what flights are available from pittsburgh to...
1,atis_flight_time,what is the arrival time in san francisco for...
2,atis_airfare,cheapest airfare from tacoma to orlando
3,atis_airfare,round trip fares from pittsburgh to philadelp...
4,atis_flight,i need a flight tomorrow from columbus to min...


In [338]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4977 entries, 0 to 4976
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Intent  4977 non-null   object
 1   Text    4977 non-null   object
dtypes: object(2)
memory usage: 77.9+ KB


In [339]:
df.Intent.value_counts()
#len(df.Intent.value_counts())

Intent
atis_flight                                 3665
atis_airfare                                 423
atis_ground_service                          255
atis_airline                                 157
atis_abbreviation                            147
atis_aircraft                                 81
atis_flight_time                              54
atis_quantity                                 51
atis_flight#atis_airfare                      21
atis_airport                                  20
atis_distance                                 20
atis_city                                     19
atis_ground_fare                              18
atis_capacity                                 16
atis_flight_no                                12
atis_meal                                      6
atis_restriction                               6
atis_airline#atis_flight_no                    2
atis_ground_service#atis_ground_fare           1
atis_airfare#atis_flight_time                  1
atis_cheapest

>Findings
- There are 4977 entries and 22 different type of intents

In [340]:
# get rid of some columns where intent counts is less than 10
#atis_flight#atis_airfare
# atis_meal
# atis_restriction
# atis_airline#atis_flight_no
# atis_ground_service#atis_ground_fare
# atis_airfare#atis_flight_time
# atis_cheapest
# atis_aircraft#atis_flight#atis_flight_no


In [341]:
# type(df.Intent.value_counts())
df.Intent.value_counts()[0:15]

Intent
atis_flight                 3665
atis_airfare                 423
atis_ground_service          255
atis_airline                 157
atis_abbreviation            147
atis_aircraft                 81
atis_flight_time              54
atis_quantity                 51
atis_flight#atis_airfare      21
atis_airport                  20
atis_distance                 20
atis_city                     19
atis_ground_fare              18
atis_capacity                 16
atis_flight_no                12
Name: count, dtype: int64

In [342]:
df.Intent.value_counts()[0:15].keys()

Index(['atis_flight', 'atis_airfare', 'atis_ground_service', 'atis_airline',
       'atis_abbreviation', 'atis_aircraft', 'atis_flight_time',
       'atis_quantity', 'atis_flight#atis_airfare', 'atis_airport',
       'atis_distance', 'atis_city', 'atis_ground_fare', 'atis_capacity',
       'atis_flight_no'],
      dtype='object', name='Intent')

In [343]:
#df=df[df.Intent.isin(df.Intent.value_counts()[0:15].keys())]
top_15_columns= df.Intent.value_counts()[0:15].keys()
data =[]
for idx,intent in enumerate(df['Intent']):
  if intent in top_15_columns:
    data.append([df['Text'][idx],df['Intent'][idx]])

df_sliced=pd.DataFrame(data,columns=['Text','Intent'])
df_sliced.head()

,Text,Intent
0,what flights are available from pittsburgh to...,atis_flight
1,what is the arrival time in san francisco for...,atis_flight_time
2,cheapest airfare from tacoma to orlando,atis_airfare
3,round trip fares from pittsburgh to philadelp...,atis_airfare
4,i need a flight tomorrow from columbus to min...,atis_flight


In [344]:
df_sliced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4959 entries, 0 to 4958
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    4959 non-null   object
 1   Intent  4959 non-null   object
dtypes: object(2)
memory usage: 77.6+ KB


# 3.Data Preprocessing on X_data and Y_data and 4. Split the dataset into X_data and Y_data
Reduce the model complexity such a way accuracy should improve withoutoverfitting or underfitting.



In [345]:
# convert unstructure data to structure format
# in Machine Learning - Vecto
# # Countvectorizer
##TFIDFVectorizer
# Deep Learning
# from skearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfVectorizer
#help(CountVectorizer)
cv= CountVectorizer(ngram_range =(1,1),
                stop_words =None,
                max_df =1.0,
                min_df =1,
              max_features =None )
X= cv.fit_transform(df_sliced['Text'])
X.shape
#4959 -- no . of rows
#859 -- for each row no. of columns

(4959, 859)

In [346]:
X.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [347]:
pd.DataFrame(X.toarray(),columns=cv.get_feature_names_out())

,0900,10,100,1000,1017,1020,1024,1026,1030,1039,...,worth,would,year,yes,yn,york,you,your,yyz,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4954,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4955,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4956,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4957,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [348]:
## removing stopwords
cv_nostop= CountVectorizer(ngram_range =(1,1),
                stop_words ="english",  ##stopwords considered as english
                max_df =1.0,
                min_df =1,
              max_features =None )
X_nostop= cv_nostop.fit_transform(df_sliced['Text'])
X_nostop.shape
# now column got 718 means (859-718) columns got removed which was stopwords

(4959, 718)

In [349]:
#X_nostop.toarray()

In [350]:
#pd.DataFrame(X.toarray(),columns=cv_nostop.get_feature_names_out())

In [351]:
##  stopwords and word should repeat minimum 2 times
cv_nostop_min2= CountVectorizer(ngram_range =(1,1),
                stop_words ="english",  ##stopwords considered as english
                max_df =1.0,
                min_df =2, # remove a word which  repeated atleast 2 times
              max_features =None )
X_nostop_min2= cv_nostop_min2.fit_transform(df_sliced['Text'])
X_nostop_min2.shape
# now column got 478 means (859-478) columns removed

(4959, 478)

In [352]:
#X_nostop_min2.toarray()

In [353]:
#pd.DataFrame(X.toarray(),columns=cv_nostop_min2.get_feature_names_out())

In [354]:
##  stopwords and word should repeat minimum 2 times
cv_nostop_min2_max= CountVectorizer(ngram_range =(1,1),
                stop_words ="english",  ##stopwords considered as english
                max_df =0.6, #remove a word which repeated 20% in the documents
                min_df =2, # remove a word which  repeated atleast 2 times
              max_features =None )
X_nostop_min2_max= cv_nostop_min2_max.fit_transform(df_sliced['Text'])
X_nostop_min2_max.shape
# now column got 478 means (859-478) columns removed

(4959, 478)

In [355]:
#X_nostop_min2_max.toarray()

In [356]:
#pd.DataFrame(X.toarray(),columns=cv_nostop_min2_max.get_feature_names_out())

In [357]:
##  select with top 250 words in text
cv_nostop_top250= CountVectorizer(ngram_range =(1,1),
                stop_words ="english",  ##stopwords considered as english
                max_df =0.6, #remove a word which repeated 60% in the documents
                min_df =2, # remove a word which  repeated atleast 2 times
              max_features =250  #select with top 250 words
                                  )
X_nostop_top250= cv_nostop_top250.fit_transform(df_sliced['Text'])
X_nostop_top250.shape
# now column got 250 means (859-250) columns removed

(4959, 250)

In [358]:
#X_nostop_top250.toarray()

In [359]:
#pd.DataFrame(X.toarray(),columns=cv_nostop_top250.get_feature_names_out())

In [360]:
##  select with bigram  words in text
cv_nostop_bigram= CountVectorizer(ngram_range =(1,2),
                stop_words ="english",  ##stopwords considered as english
              max_features =None
                                  )
X_nostop_bigram= cv_nostop_bigram.fit_transform(df_sliced['Text'])
X_nostop_bigram.shape
# now column got 6015 means (859-6015) columns removed

(4959, 6015)

In [361]:
#X_nostop_bigram.toarray()

In [362]:
#pd.DataFrame(X.toarray(),columns=cv_nostop_bigram.get_feature_names_out())

In [363]:
cv_nostop_bigram.get_feature_names_out()[20:30]

array(['1024', '1024 morning', '1026', '1030', '1030 1130', '1039',
       '1039 denver', '1039 thursday', '1045', '1055'], dtype=object)

In [364]:
#from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y_data=le.fit_transform(df_sliced['Intent'])
y_data

array([ 8, 11,  2, ...,  3,  8,  8])

# 5.Split the data into training and testing and 6. Use only training data to train the model

In [ ]:
#cv_nostop_min2
X_train,X_test,y_train,y_test=train_test_split(cv_nostop_min2,
                                               y_data,test_size=0.2,
                                               random_state=42)